# 数据导入

In [ ]:
#//导入需要的包
from pyspark.ml.feature import PCA 
from pyspark.sql import Row
from pyspark.ml.linalg import Vector,Vectors
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline,PipelineModel
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer,HashingTF, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.ml.classification import BinaryLogisticRegressionSummary, LogisticRegression
from pyspark.sql import functions
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


## 基本处理



首先对训练集和测试集做一个基本处理，不然会报错：

1.去掉测试集首行：|1x3 Cross validator 不然报错
ValueError: could not convert string to float: '|1x3 Cross validator'

2.获取训练集测试集（需要对测试集进行一下处理，adult.data.txt的标签是>50K和<=50K，
而adult.test.txt的标签是>50K.和<=50K.，这里是把adult.test.txt标签的“.”去掉了。另外，确保adult.data.txt和adult.test.txt最后没有多一个空
格。）

In [16]:
import pyspark
from pyspark.ml.feature import CountVectorizer
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark import SparkConf, SparkContext



try:
    spark.stop()
    print("Stopped a SparkSession")
except Exception as e:
    print("No existing SparkSession")
conf = SparkConf().setMaster("local").setAppName("My App")

sc = SparkContext(conf = conf)
spark = SparkSession(sc)


Stopped a SparkSession


In [17]:
def f(x):
    rel = {}
    rel['features']=Vectors.dense(float(x[0]),float(x[2]),float(x[4]),float(x[10]),float(x[11]),float(x[12]))
    rel['label'] = str(x[14])
    return rel

In [18]:
df = spark.sparkContext.textFile("file:///data/season/adult.data.txt").map(lambda line: line.split(',')).map(lambda p: Row(**f(p))).toDF() 

In [61]:
text_rdd = spark.sparkContext.textFile("file:///data/season/adult.data.txt")
text_rdd

file:///data/season/adult.data.txt MapPartitionsRDD[14601] at textFile at NativeMethodAccessorImpl.java:0

In [19]:

test = spark.sparkContext.textFile("file:///data/season//adult.test.txt").map(lambda line: line.split(',')).map(lambda p: Row(**f(p))).toDF()

# 2.进行主成分分析（PCA）
对6个连续型的数值型变量进行主成分分析。

PCA（主成分分析）是通过正交变换把一组相关变量的观测值转化成一组线性无关的变量值，即主成分的一种方法。
PCA通过使用主成分把特征向量投影到低维空间，实现对特征向量的降维。请通过setK()方法将主成分数量设置为3，把连续型的特征向量转化成一个3维的主成分。


构建PCA模型，并通过训练集进行主成分分解，然后分别应用到训练集和测试集


In [23]:
pca = PCA(k=3, inputCol="features", outputCol="pcaFeatures").fit(df)


In [24]:
result = pca.transform(df)
testdata = pca.transform(test)


In [25]:
result.show(truncate=False)

+------------------------------------+------+-----------------------------------------------------------+
|features                            |label |pcaFeatures                                                |
+------------------------------------+------+-----------------------------------------------------------+
|[39.0,77516.0,13.0,2174.0,0.0,40.0] | <=50K|[77516.06543281932,-2171.6489938846585,-6.946360476597975] |
|[50.0,83311.0,13.0,0.0,0.0,13.0]    | <=50K|[83310.99935595779,2.526033892791933,-3.3887024086795283]  |
|[38.0,215646.0,9.0,0.0,0.0,40.0]    | <=50K|[215645.99925048652,6.551842584549856,-8.584953969072847]  |
|[53.0,234721.0,7.0,0.0,0.0,40.0]    | <=50K|[234720.99907961808,7.130299808617081,-9.36017979080908]   |
|[28.0,338409.0,13.0,0.0,0.0,40.0]   | <=50K|[338408.99918830546,10.289249842815364,-13.368251871630113]|
|[37.0,284582.0,14.0,0.0,0.0,40.0]   | <=50K|[284581.99916695454,8.649756033709732,-11.281731333792491] |
|[49.0,160187.0,5.0,0.0,0.0,16.0]    | <=50K|[

In [26]:
testdata.show(truncate=False) 

+------------------------------------+------+-----------------------------------------------------------+
|features                            |label |pcaFeatures                                                |
+------------------------------------+------+-----------------------------------------------------------+
|[25.0,226802.0,7.0,0.0,0.0,40.0]    | <=50K|[226801.9993670891,6.8933130423286935,-8.99398382175794]   |
|[38.0,89814.0,9.0,0.0,0.0,50.0]     | <=50K|[89813.9993894769,2.720987324477724,-3.6809508659700634]   |
|[28.0,336951.0,12.0,0.0,0.0,40.0]   | >50K |[336950.9991912231,10.244920104030937,-13.310695651854761] |
|[44.0,160323.0,10.0,7688.0,0.0,40.0]| >50K |[160323.2327290343,-7683.121090489606,-19.729118648468784] |
|[18.0,103497.0,10.0,0.0,0.0,30.0]   | <=50K|[103496.99961293538,3.142862309152393,-4.141563083945909]  |
|[34.0,198693.0,6.0,0.0,0.0,30.0]    | <=50K|[198692.99933690467,6.037911774656125,-7.894879761308832]  |
|[29.0,227026.0,9.0,0.0,0.0,40.0]    | <=50K|[

# 3.训练分类模型并预测居民收入

在主成分分析的基础上，采用逻辑斯蒂回归，或者决策树模型预测居民收入是否超过50K；对Test数据集进行验证。

训练逻辑斯蒂回归模型，并进行测试，得到预测准确率



In [28]:
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(result)

In [29]:
for label in labelIndexer.labels:print(label)

 <=50K
 >50K


In [30]:
featureIndexer = VectorIndexer(inputCol="pcaFeatures", outputCol="indexedFeatures").fit(result)
print(featureIndexer.numFeatures)

3


In [31]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",labels=labelIndexer.labels)

In [32]:
lr = LogisticRegression().setLabelCol("indexedLabel").setFeaturesCol("indexedFeatures").setMaxIter(100)

In [33]:
lrPipeline = Pipeline().setStages([labelIndexer, featureIndexer, lr, labelConverter])

In [34]:
lrPipelineModel = lrPipeline.fit(result)
lrModel = lrPipelineModel.stages[2]
print ("Coefficients: \n " + str(lrModel.coefficientMatrix)+"\nIntercept: "+str(lrModel.interceptVector)+ "\n numClasses: "+str(lrModel.numClasses)+"\n numFeatures: "+str(lrModel.numFeatures))


Coefficients: 
 DenseMatrix([[-1.98285864e-07, -3.50909247e-04, -8.45150628e-04]])

Intercept: [-1.4525982557843347]
 numClasses: 2
 numFeatures: 3


In [40]:
lrPredictions = lrPipelineModel.transform(testdata)
lrPredictions

DataFrame[features: vector, label: string, pcaFeatures: vector, indexedLabel: double, indexedFeatures: vector, rawPrediction: vector, probability: vector, prediction: double, predictedLabel: string]

In [41]:
evaluator = MulticlassClassificationEvaluator().setLabelCol("indexedLabel").setPredictionCol("prediction")
evaluator

MulticlassClassificationEvaluator_c2198bf9aaa6

In [42]:
lrAccuracy = evaluator.evaluate(lrPredictions)
lrAccuracy

0.7764235163053484

In [39]:
print("Test Error = %g " % (1.0 - lrAccuracy))

Test Error = 0.223576 


# 4.超参数调优
利用CrossValidator确定最优的参数，包括最优主成分PCA的维数、分类器自身的参数等。




In [43]:
pca = PCA().setInputCol("features").setOutputCol("pcaFeatures")

In [44]:
labelIndexer = StringIndexer().setInputCol("label").setOutputCol("indexedLabel").fit(df)
labelIndexer

StringIndexerModel: uid=StringIndexer_20711d1a8681, handleInvalid=error

In [45]:
featureIndexer = VectorIndexer().setInputCol("pcaFeatures").setOutputCol("indexedFeatures")
featureIndexer

VectorIndexer_f390a1647067

In [46]:
labelConverter = IndexToString().setInputCol("prediction").setOutputCol("predictedLabel").setLabels(labelIndexer.labels)
labelConverter

IndexToString_55acf7adaf58

In [47]:
lr = LogisticRegression().setLabelCol("indexedLabel").setFeaturesCol("indexedFeatures").setMaxIter(100)
lr

LogisticRegression_9e679e254021

In [48]:
lrPipeline = Pipeline().setStages([pca, labelIndexer, featureIndexer, lr, labelConverter])
lrPipeline

Pipeline_426c31bb86ff

In [49]:
paramGrid = ParamGridBuilder().addGrid(pca.k, [1,2,3,4,5,6]).addGrid(lr.elasticNetParam, [0.2,0.8]).addGrid(lr.regParam, [0.01, 0.1, 0.5]).build()
paramGrid

[{Param(parent='PCA_572e915517ec', name='k', doc='the number of principal components'): 1,
  Param(parent='LogisticRegression_9e679e254021', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.2,
  Param(parent='LogisticRegression_9e679e254021', name='regParam', doc='regularization parameter (>= 0).'): 0.01},
 {Param(parent='PCA_572e915517ec', name='k', doc='the number of principal components'): 1,
  Param(parent='LogisticRegression_9e679e254021', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.2,
  Param(parent='LogisticRegression_9e679e254021', name='regParam', doc='regularization parameter (>= 0).'): 0.1},
 {Param(parent='PCA_572e915517ec', name='k', doc='the number of principal components'): 1,
  Param(parent='LogisticRegression_9e679e254021', name='elasti

In [50]:
cv = CrossValidator().setEstimator(lrPipeline).setEvaluator(MulticlassClassificationEvaluator().setLabelCol("indexedLabel").setPredictionCol("prediction")).setEstimatorParamMaps(paramGrid).setNumFolds(3)
cv



CrossValidator_92dee98a02a6

In [51]:
cvModel = cv.fit(df)
cvModel

CrossValidatorModel_7e4491e29333

In [52]:
lrPredictions=cvModel.transform(test)
lrPredictions

DataFrame[features: vector, label: string, pcaFeatures: vector, indexedLabel: double, indexedFeatures: vector, rawPrediction: vector, probability: vector, prediction: double, predictedLabel: string]

In [53]:
evaluator = MulticlassClassificationEvaluator().setLabelCol("indexedLabel").setPredictionCol("prediction")
evaluator

MulticlassClassificationEvaluator_eed2d879befc

In [54]:
lrAccuracy = evaluator.evaluate(lrPredictions)
lrAccuracy

0.7833268290041506

In [60]:
print("准确率为"+str(lrAccuracy))

准确率为0.7833268290041506


In [55]:
bestModel= cvModel.bestModel
bestModel

PipelineModel_1f7cee47b3d4

In [56]:
lrModel = bestModel.stages[3]
lrModel

LogisticRegressionModel: uid=LogisticRegression_9e679e254021, numClasses=2, numFeatures=6

In [57]:
print ("Coefficients: \n " + str(lrModel.coefficientMatrix)+"\nIntercept: "+str(lrModel.interceptVector)+ "\n numClasses: "+str(lrModel.numClasses)+"\n numFeatures: "+str(lrModel.numFeatures))

Coefficients: 
 DenseMatrix([[-1.50035172e-07, -1.68933655e-04, -8.83869475e-04,
               4.92262006e-02,  3.10992712e-02, -2.81742804e-01]])
Intercept: [-7.459195847829065]
 numClasses: 2
 numFeatures: 6


In [58]:
pcaModel = bestModel.stages[0]
pcaModel

PCAModel: uid=PCA_572e915517ec, k=6

In [59]:
print("Primary Component: " + str(pcaModel.pc))

Primary Component: DenseMatrix([[-9.90507714e-06, -1.43514070e-04, -2.01710563e-03,
               9.62889111e-01, -2.69881464e-01,  2.11340373e-03],
             [ 9.99999999e-01,  3.04337871e-05, -3.91106215e-05,
               1.00509067e-05, -5.73044237e-07, -9.50170012e-07],
             [-1.05283840e-06, -4.27228452e-05, -5.32933664e-04,
               9.87277274e-03,  2.74007689e-02, -9.99575630e-01],
             [ 3.03678811e-05, -9.99998483e-01, -1.72935042e-03,
              -1.78145850e-04, -9.05936805e-05,  3.94199762e-05],
             [-3.91389877e-05,  1.72989546e-03, -9.99994820e-01,
              -2.41554925e-03, -1.14142219e-03,  4.77935846e-04],
             [-2.19555372e-06, -1.31095844e-04, -1.73646184e-03,
               2.69705807e-01,  9.62502926e-01,  2.90493230e-02]])


可以看出，PCA最优的维数是6。